In [1]:
import json
from pathlib import Path
import shutil

import pandas as pd
import numpy as np

from bw2io import remote,create_default_lcia_methods
from bw2io.importers.io import IOImporter
from bw2io.strategies.io import tidy_tables
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba

In [2]:
print(bd.__version__)

(4, 5)


## define the database

In [4]:
pet_hiot = pd.DataFrame([[1,-2],[0,1]], # experiment setting 1 prod to 0
             index=pd.MultiIndex.from_tuples([('DK','prod1'),('DK','prod2')]),
             columns = pd.MultiIndex.from_tuples([('DK','act1'),('DK','act2')]))

pet_hiot = pet_hiot.astype(pd.SparseDtype("float",0))


B = pd.DataFrame([[1,3],[1,2],[0,1],[4,0]],
             index=pd.Index(['co2_air','ch4_air','co2_accelerated_air','land_occupation']),
             columns = pd.MultiIndex.from_tuples([('DK','act1'),('DK','act2')]))

B = B.astype(pd.SparseDtype("float",0))

fd = pd.DataFrame.from_dict(
    {
    ('DK','Household'):{('DK','prod1'):-11,('DK','prod2'):-3},
    ('DK','Government'):{('DK','prod1'):-8,('DK','prod2'):-4},
    ('DK','Capital'):{('DK','prod1'):-4,('DK','prod2'):-2}
        }
    )

Bfd = pd.DataFrame([[1,3],],
index=pd.Index(['co2_air',]),
columns = pd.MultiIndex.from_tuples([('DK','Household'),
('DK','Government')])).astype(pd.SparseDtype("float",0))

In [5]:
print(pet_hiot)
print(B)
print(fd)
print(Bfd)

           DK     
         act1 act2
DK prod1  1.0 -2.0
   prod2    0  1.0
                      DK     
                    act1 act2
co2_air              1.0  3.0
ch4_air              1.0  2.0
co2_accelerated_air    0  1.0
land_occupation      4.0    0
                DK                   
         Household Government Capital
DK prod1       -11         -8      -4
   prod2        -3         -4      -2
               DK           
        Household Government
co2_air       1.0        3.0


In [6]:
pfd = pd.DataFrame((np.eye(fd.shape[1])),index=fd.columns,columns=fd.columns)
fd_total = pd.concat([fd,pfd])
fd_total = fd_total.astype(pd.SparseDtype("float",0))

In [7]:
extended_hiot = pd.concat([pet_hiot,fd_total],axis=1).fillna(0)
extended_B = pd.concat([B,Bfd],axis=1).fillna(0)

In [8]:
print(extended_hiot)

                DK                                  
              act1 act2 Household Government Capital
DK prod1       1.0 -2.0     -11.0       -8.0    -4.0
   prod2         0  1.0      -3.0       -4.0    -2.0
   Household   0.0  0.0       1.0          0       0
   Government  0.0  0.0         0        1.0       0
   Capital     0.0  0.0         0          0     1.0


In [9]:
print(extended_B)

                      DK                          
                    act1 act2 Household Government
co2_air              1.0  3.0       1.0        3.0
ch4_air              1.0  2.0       0.0        0.0
co2_accelerated_air    0  1.0       0.0        0.0
land_occupation      4.0    0       0.0        0.0


In [10]:
path_to_intermediate = (Path.cwd()/'results')
path_to_intermediate.mkdir(exist_ok=True)

In [11]:
metadata_dict = {'prod1':{'unit':'kg','name':'product 1'},
                 'prod2':{'unit':'kg','name':'product 2'},
                 'Household':{'unit':'unit','name':'the household'},
                 'Government':{'unit':'unit','name':'the government'},
                 'Capital':{'unit':'unit','name':'capital investments'},

                 'co2_air':{'unit':'ton', # not standard units
                            'name':'carbon dioxide',
                            'compartment':('air',)},
                 'ch4_air':{'unit':'kg',
                            'name':'methane',
                            'compartment':('air',)},
                 'co2_accelerated_air':{'unit':'kg', # additional biosphere flow
                                        'name':'carbon dioxide accelerated',
                                        'compartment':('air',)},
                 'land_occupation':{'unit':'hectare * year', # non standard composite unit
                                    'name':'land occupation',
                                    'compartment':('natural resource', 'land')}
                }

with open(path_to_intermediate/'io_metadata.json', 'w') as fp:
    json.dump(metadata_dict, fp,indent=4)

In [12]:
tidy_tables(extended_hiot,extended_B,path_to_intermediate)

In [13]:
#bd.projects.set_current('test_io_importer','test_io_importer')

In [14]:
remote.install_project('ecoinvent-3.10-biosphere','test_io_importer',
                       overwrite_existing=False)

Restoring project backup archive - this could take a few minutes...


ValueError: Project test_io_importer already exists, set `overwrite_existing=True` to overwrite

In [15]:
bd.projects.set_current('test_io_importer')

In [74]:
bd.methods

Methods dictionary with 668 objects, including:
	('CML v4.8 2016', 'acidification', 'acidification (incl. fate, average Europe total, A&B)')
	('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')
	('CML v4.8 2016', 'ecotoxicity: freshwater', 'freshwater aquatic ecotoxicity (FAETP inf)')
	('CML v4.8 2016', 'ecotoxicity: marine', 'marine aquatic ecotoxicity (MAETP inf)')
	('CML v4.8 2016', 'ecotoxicity: terrestrial', 'terrestrial ecotoxicity (TETP inf)')
	('CML v4.8 2016', 'energy resources: non-renewable', 'abiotic depletion potential (ADP): fossil fuels')
	('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)')
	('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)')
	('CML v4.8 2016', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)')
	('CML v4.8 2016', 'ozone depletion', 'ozone layer depletion (ODP steady state)')
Use `list(this object)` to get the complete list.

In [18]:
try:
    del bd.databases['pet_io_db_1 biosphere']
    del bd.databases['pet_io_db_1']
    bd.Database('pet_io_db_1').delete(warn=False)
    bd.Database('pet_io_db biosphere_1').delete(warn=False)
except KeyError:
    print('db not there')

In [19]:
bd.databases

Databases dictionary with 1 object(s):
	ecoinvent-3.10-biosphere

These include the links between the database being imported and the biosphere3 database. 

For those that do not have a correspondence, a new elementary flow will be created in an additional biosphere db

In [ ]:
co2 = bd.Database(bd.config.biosphere).get(name='Carbon dioxide, fossil',categories=('air',))
ch4 = bd.Database(bd.config.biosphere).get(name='Methane, fossil',categories=('air',))
land_occupation = bd.Database(bd.config.biosphere).get(name='Occupation, unspecified',
                                                       categories=('natural resource', 'land'))

In [59]:
biosphere_mapping = {
    'co2_air':co2['code'],
    'ch4_air':ch4['code'],
    'land_occupation':land_occupation['code'],
    }

In [21]:
pet_example = IOImporter(path_to_intermediate,'pet_io_db',b3mapping=dict())
pet_example.apply_strategies()

In [22]:
pet_example.write_database()

100%|██████████| 4/4 [00:00<00:00, 34169.48it/s]

18:04:59+0100 [info     ] Vacuuming database            



100%|██████████| 5/5 [00:00<00:00, 1897.02it/s]

18:04:59+0100 [info     ] Vacuuming database            


18:04:59+0100 [info     ] Starting IO table write       
18:04:59+0100 [info     ] Adding technosphere matrix    
18:04:59+0100 [info     ] Adding biosphere matrix       
18:04:59+0100 [info     ] Finalizing serialization      


In [6]:
for b in bd.Database('pet_io_db biosphere'):
    print(b)

'land occupation' (hectare * year, None, ('natural resource', 'land'))
'carbon dioxide accelerated' (kilogram, None, ('air',))


## test

In [23]:
test_db = bd.Database('pet_io_db')

In [24]:
act1 = test_db.get(code='act1|DK')
act2 = test_db.get(code='act2|DK')

In [25]:
# it should have CO2 emissions close to 1 ton and land use of 40000 m2 * year
for e in act1.biosphere():
    print(e)

Exchange: 1.0 kilogram 'methane' (kilogram, None, ('air',)) to 'act1' (kilogram, DK, None)>
Exchange: 4.0 hectare * year 'land occupation' (hectare * year, None, ('natural resource', 'land')) to 'act1' (kilogram, DK, None)>
Exchange: 1.0 ton 'carbon dioxide' (ton, None, ('air',)) to 'act1' (kilogram, DK, None)>


In [75]:
ipcc_2021 = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
assert ipcc_2021 in bd.methods

In [61]:
test_lca = bc.LCA({act2:1},ipcc_2021)

In [62]:
test_lca.lci()
test_lca.lcia()
test_lca.score

/home/miguel/miniconda3/envs/ds/lib/python3.12/site-packages/bw2calc/lca.py:247: UserWarning: All values in characterization matrix are zero
  warnings.warn("All values in characterization matrix are zero")


0.0

In [63]:
extended_hiot

DK                                  
              act1 act2 Household Government Capital
DK prod1       1.0 -2.0     -11.0       -8.0    -4.0
   prod2         0  1.0      -3.0       -4.0    -2.0
   Household   0.0  0.0       1.0          0       0
   Government  0.0  0.0         0        1.0       0
   Capital     0.0  0.0         0          0     1.0

In [29]:
test_lca.technosphere_matrix.todense()

matrix([[  1.,  -2., -11.,  -8.,  -4.],
        [  0.,   1.,  -3.,  -4.,  -2.],
        [  0.,   0.,   1.,   0.,   0.],
        [  0.,   0.,   0.,   1.,   0.],
        [  0.,   0.,   0.,   0.,   1.]])

In [64]:
B = pd.DataFrame(test_lca.biosphere_matrix.todense())
B = B.rename(test_lca.biosphere_dict.reversed)
B.index = B.index.map(lambda x:bd.get_node(id=x)).map(lambda x:(x['name'],x['unit']))

In [66]:
B = B.rename(test_lca.activity_dict.reversed,axis=1)
B.columns = B.columns.map(lambda x:bd.get_node(id=x)).map(lambda x:(x['name'],x['unit']))

In [68]:
B

,,act1,act2,the household,the government,capital investments
,,kilogram,kilogram,unit,unit,unit
methane,kilogram,1.0,2.0,0.0,0.0,0.0
land occupation,hectare * year,4.0,0.0,0.0,0.0,0.0
carbon dioxide accelerated,kilogram,0.0,1.0,0.0,0.0,0.0
carbon dioxide,ton,1.0,3.0,1.0,3.0,0.0


In [70]:
test_lca.characterization_matrix.todense()

matrix([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [67]:
ba.print_recursive_calculation(act2,ipcc_2021,use_matrix_values=True)

Fraction of score | Absolute score | Amount | Activity


/home/miguel/miniconda3/envs/ds/lib/python3.12/site-packages/bw2calc/lca.py:247: UserWarning: All values in characterization matrix are zero
  warnings.warn("All values in characterization matrix are zero")


ZeroDivisionError: float division by zero

## compatibility with bw_aggregation 

In [30]:
test_db.backend

'iotable'

In [31]:
import bw_aggregation

In [32]:
bw_aggregation.AggregatedDatabase.convert_existing(test_db.name)

Starting inventory calculation. Please be patient, we have 5 processes to calculate.


In [33]:
bd.Database('pet_io_db')

Brightway2 AggregatedDatabase: pet_io_db

In [34]:
bd.Database('pet_io_db').use_aggregated(False)

In [35]:
act2 = bd.Database('pet_io_db').get(code='act2|DK')

In [36]:
ba.print_recursive_calculation(act2,ipcc_2021)

Fraction of score | Absolute score | Amount | Activity
0001 |  4655 |     1 | 'act2' (kilogram, DK, None)


In [37]:
with bw_aggregation.AggregationContext(False):

    ba.print_recursive_calculation(act2,ipcc_2021)

Fraction of score | Absolute score | Amount | Activity
0001 |  4655 |     1 | 'act2' (kilogram, DK, None)


In [38]:
lca = act2.lca(ipcc_2021)

In [41]:
# if true, then it is not aggregated..
'col_code' in lca.to_dataframe().columns

True

exchanges are no longer there

In [45]:
list(act2.exchanges())

[]